In [ ]:
## download the EM-EARTH data and combine it to the model simulation

import os
import requests
import base64
import hashlib
import glob
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import os

def download_EM_ERATH (server = 'https://g-772fa5.cd4fe.0ec8.data.globus.org/6/published/publication_542/submitted_data/',
                       groups = ['deterministic_hourly','deterministic_raw_daily','probabilistic_daily'],
                       variables = ['prcp','tdew','tmean','trange'],
                       regions = ['Asia','Europe','Africa','NorthAmerica','Oceania','SouthAmerica'],
                       years = ['1950','1951','1952','1953','1954',\
                                '1955','1956','1957','1958','1959',\
                                '1960','1961','1962','1963','1964',\
                                '1965','1966','1967','1968','1969',\
                                '1970','1971','1972','1973','1974',\
                                '1975','1976','1977','1978','1979',\
                                '1980','1981','1982','1983','1984',\
                                '1985','1986','1987','1988','1989',\
                                '1990','1991','1992','1993','1994',\
                                '1995','1996','1997','1998','1999',\
                                '2000','2001','2002','2003','2004',\
                                '2005','2006','2007','2008','2009',\
                                '2010','2011','2012','2013','2014',\
                                '2015','2016','2017','2018','2019'],
                       months = ['01','02','03','04','05','06',\
                                 '07','08','09','10','11','12'],
                       ensembles = ['001','002','003','004','005',\
                                    '006','007','008','009','010',\
                                    '011','012','013','014','015',\
                                    '016','017','018','019','020',\
                                    '021','022','023','024','025'],
                       root_save = '../EM_Earth_v1/'):
    
    
    for group in groups:
        
        for year in years:
            
            for month in months:
                
                for variable in variables:
                    
                    if group == 'deterministic_raw_daily':
                        
                        file_name = 'EM_Earth_deterministic_daily_'+variable+'_'+year+month+'.nc'
                        path      = group+'/'+variable+'/'
                        link      = server+'EM_Earth_v1/'+path+file_name
                        path_save = root_save + path
                        if not os.path.isdir(path_save):
                            os.makedirs(path_save)
                        download_file (link,path_save,file_name)
                    
                    for region in regions:
                            
                            if group == 'deterministic_hourly' and variable != 'trange':
                                
                                file_name = 'EM_Earth_'+group+'_'+region+'_'+year+month+'.nc'
                                path      = group+'/'+variable+'/'+region+'/'
                                link      = server+'EM_Earth_v1/'+path+file_name
                                path_save = root_save + path
                                if not os.path.isdir(path_save):
                                    os.makedirs(path_save)
                                download_file (link,path_save,file_name)
                        
                            
                            if group == 'probabilistic_daily':

                                for ensemble in ensembles:

                                    file_name = 'EM_Earth_probabilistic_daily_'+variable+'_'+region+'_'+\
                                    year+month+'_'+ensemble+'.nc'
                                    path      = group+'/'+variable+'/'+region+'/'+year+month+'/'
                                    link      = server+'EM_Earth_v1/'+path+file_name
                                    path_save = root_save + path
                                    if not os.path.isdir(path_save):
                                        os.makedirs(path_save)
                                    download_file (link,path_save,file_name)


def download_file (link,path_save,file_name):
    
    
    os.system('rm *.nc*')
    downloaded = False
    try_number = 1
    while (not downloaded) and (try_number < 1000):
        print('going to download '+file_name + ': '+str(try_number))
        os.system('wget -nv '+link +' '+file_name)
        #os.system('wget -O - link | tee file_name | md5sum > remote')
        os.system('wget -O - '+link +' | md5sum > remote')
        os.system('md5sum '+file_name+' > local')
        md5_local = word_file('local')
        md5_remote = word_file('remote')
        print(md5_local, md5_remote)
        if md5_local == md5_remote:
            print('files are the same')
            downloaded = True
            os.system('mv '+ file_name +' '+path_save+file_name) # maybe test with mv
        try_number = try_number + 1
        os.system('rm *.nc*')


def word_file(f):
    with open(f) as file:
        return file.read().split()[0]

def merging_EM_ERATH  (variables = ['prcp','tdew','tmean','trange'],
                       regions = ['Asia','Europe','Africa','NorthAmerica','Oceania','SouthAmerica'],
                       years = ['1950','1951','1952','1953','1954',\
                                '1955','1956','1957','1958','1959',\
                                '1960','1961','1962','1963','1964',\
                                '1965','1966','1967','1968','1969',\
                                '1970','1971','1972','1973','1974',\
                                '1975','1976','1977','1978','1979',\
                                '1980','1981','1982','1983','1984',\
                                '1985','1986','1987','1988','1989',\
                                '1990','1991','1992','1993','1994',\
                                '1995','1996','1997','1998','1999',\
                                '2000','2001','2002','2003','2004',\
                                '2005','2006','2007','2008','2009',\
                                '2010','2011','2012','2013','2014',\
                                '2015','2016','2017','2018','2019'],
                       months = ['01','02','03','04','05','06',\
                                 '07','08','09','10','11','12'],
                       ensembles = ['001','002','003','004','005',\
                                    '006','007','008','009','010',\
                                    '011','012','013','014','015',\
                                    '016','017','018','019','020',\
                                    '021','022','023','024','025'],
                       root_save = '../EM_Earth_v1/'):
    
    
    for year in years:

        for month in months:

            for variable in variables:
                
                # file_name
                file_name = root_save+'deterministic_raw_daily/'+variable+'/'+\
                'EM_Earth_deterministic_daily_'+variable+'_'+year+month+'.nc'
                
                # open dataset
                ds_global = xr.open_dataset(file_name)
                #print(ds_global.lon[0:3])
                
                # reorganize the variables to time, lat, lon
                ds_global = ds_global.transpose('time', 'lat', 'lon')
                
                mask_sum ={}
                # replace the varibales with 0.00
                for v in ds_global.data_vars:
                    ds_global [v][:] = 0.00
                    mask_sum[v] = np.zeros([1800, 3600])
                
                # loop over the ensembles
                for ensemble in ensembles:
                    
                    #  read the files
                    file_names = root_save+'probabilistic_daily/'+variable+'/*/'+\
                                           year+month+'/*'+ensemble+'.nc'
                    file_names = glob.glob(file_names)
                    
                    #
                    for file_name in file_names:
                        print(file_name)
                        ds = xr.open_dataset(file_name)
                        #print(ds.data_vars)
                        ds = ds.transpose('time', 'lat', 'lon') # making sure the order
                        
                        if ds.lon.values.max() > 180:
                            # Asia
                            ds1 = ds.sel(lon=slice(None, 179.975))
                            ds2 = ds.sel(lon=slice(179.975, None))
                            ds2['lon'] = ds2['lon'] - 360
                            ds_global, mask_sum = fill_cont_to_globe(ds1, ds_global, mask_sum)
                            ds_global, mask_sum = fill_cont_to_globe(ds2, ds_global, mask_sum)
                        elif ds.lon.values.min() < -180:
                            # North America
                            ds1 = ds.sel(lon=slice(None, -179.975))
                            ds2 = ds.sel(lon=slice(-179.975, None))
                            ds1['lon'] = ds1['lon'] + 360
                            ds_global, mask_sum = fill_cont_to_globe(ds1, ds_global, mask_sum)
                            ds_global, mask_sum = fill_cont_to_globe(ds2, ds_global, mask_sum)
                        else:
                            ds_global, mask_sum = fill_cont_to_globe(ds , ds_global, mask_sum)
                    
                    
                    file_name = 'EM_Earth_probabilistic_daily_'+variable+'_global_'+\
                                year+month+'_'+ensemble+'.nc'
                    folder_name = root_save+'probabilistic_daily_global/'+variable+'/'+\
                                year+month+'/'
                    if not os.path.isdir(folder_name):
                        os.makedirs(folder_name)
                    
                    for name, value in ds_global.items():
                        for t in range(value.shape[0]):
                            temp = np.array(value[t,:,:]) / mask_sum[name]
                            temp[mask_sum[name] == 0] = -9999
                            value[t, :, :] = temp
                        ds_global[name] = value
                    
                    print(file_name)
                    
                    os.system('rm '+folder_name+file_name)
                    
                    encoding = {}
                    for var in ds_global.data_vars:
                        encoding[var] = {'_FillValue': -9999., 'zlib': True, 'complevel': 4}
                        
                    ds_global.to_netcdf(folder_name+file_name, unlimited_dims='time', encoding=encoding)
                    
                    


def fill_cont_to_globe(ds, ds_global, mask_sum):
    
    row_start = np.where(np.array(ds_global.lat[:]) == ds.lat.max().item())
    row_end   = np.where(np.array(ds_global.lat[:]) == ds.lat.min().item())
    col_start = np.where(np.array(ds_global.lon[:]) == ds.lon.min().item())
    col_end   = np.where(np.array(ds_global.lon[:]) == ds.lon.max().item())
    row_start = row_start[0].item()
    row_end   = row_end[0].item()
    col_start = col_start[0].item()
    col_end   = col_end[0].item()
    
    print(row_start, row_end, col_start, col_end)
    
    for v in ds.data_vars:
        
        #
        datav = ds[v].values.copy()
        datav[np.isnan(datav)] = 0 # replace NaN with 0
        ds_global[v][:, row_start:row_end+1, col_start:col_end+1] = \
        ds_global[v][:, row_start:row_end+1, col_start:col_end+1] + datav

        #
        datav = ds[v].values.copy()
        datav = np.array(datav[0,:,:])

        datav[~np.isnan(datav)] = 1
        datav[np.isnan(datav)] = 0
        mask_sum[v][row_start:row_end+1, col_start:col_end+1] = \
        mask_sum[v][row_start:row_end+1, col_start:col_end+1] + datav
        
    
    return ds_global, mask_sum 

In [ ]:
# Download the files from HTTP
download_EM_ERATH(years = ['1950','2019'],
                  months = ['01','02'],
                  ensembles = ['001','002'])

In [ ]:
# merge the files
merging_EM_ERATH(years = ['1950','2019'],
                 months = ['01','02'],
                 ensembles = ['001','002'])

In [ ]:
# Test

# ds_global = xr.open_dataset('../EM_Earth_v1/probabilistic_daily_global/prcp/200001/EM_Earth_probabilistic_daily_prcp_global_200001_001.nc')
# ds_global = ds_global.sel(time=slice('2000-01-01'))
# ds_global = ds_global.squeeze()
# ds_global = ds_global['prcp'].to_dataframe()
# ds_global = ds_global.reset_index()
# ds_global = ds_global.drop(columns=['time'])
# ds_global['lat'] = ds_global['lat'].round(2)
# ds_global['lon'] = ds_global['lon'].round(2)
# print(ds_global)


# ds = xr.open_dataset('../EM_Earth_v1/probabilistic_daily/prcp/NorthAmerica/200001/EM_Earth_probabilistic_daily_prcp_NorthAmerica_200001_001.nc')
# ds = ds.sel(time=slice('2000-01-01'))
# ds = ds.squeeze()
# ds = ds['prcp'].to_dataframe()
# ds = ds.reset_index()
# ds = ds.drop(columns=['time'])
# ds['lat'] = ds['lat'].round(2)
# ds['lon'] = ds['lon'].round(2)
# print(ds)


# import pandas as pd
# ds_new = pd.merge(ds_global, ds,  how='right', left_on=['lat','lon'], right_on = ['lat','lon'])
# ds_new['diff'] = ds_new['prcp_x']-ds_new['prcp_y']
# print(ds_new['diff'].abs().sum())
# print(ds_new)


# ds_new['prcp_x'].plot()
# ds_new['prcp_y'].plot()
# ds_new['diff'].plot()


